In [ ]:
import tensorflow as tf
	from tensorflow.contrib import slim as contrib_slim
	
	from object_detection.meta_architectures import faster_rcnn_meta_arch
	from object_detection.utils import variables_helper
	from nets import inception_resnet_v2
	
	slim = contrib_slim
	
	
	class FasterRCNNInceptionResnetV2FeatureExtractor(
	    faster_rcnn_meta_arch.FasterRCNNFeatureExtractor):
	  """Faster R-CNN with Inception Resnet v2 feature extractor implementation."""
	
	  def __init__(self,
	               is_training,
	               first_stage_features_stride,
	               batch_norm_trainable=False,
	               reuse_weights=None,
	               weight_decay=0.0):

	    if first_stage_features_stride != 8 and first_stage_features_stride != 16:
	      raise ValueError('`first_stage_features_stride` must be 8 or 16.')
	    super(FasterRCNNInceptionResnetV2FeatureExtractor, self).__init__(
	        is_training, first_stage_features_stride, batch_norm_trainable,
	        reuse_weights, weight_decay)
	
	  def preprocess(self, resized_inputs):

	    return (2.0 / 255.0) * resized_inputs - 1.0
	
	  def _extract_proposal_features(self, preprocessed_inputs, scope):

	    if len(preprocessed_inputs.get_shape().as_list()) != 4:
	      raise ValueError('`preprocessed_inputs` must be 4 dimensional, got a '
	                       'tensor of shape %s' % preprocessed_inputs.get_shape())
	
	    with slim.arg_scope(inception_resnet_v2.inception_resnet_v2_arg_scope(
	        weight_decay=self._weight_decay)):
	      # Forces is_training to False to disable batch norm update.
	      with slim.arg_scope([slim.batch_norm],
	                          is_training=self._train_batch_norm):
	        with tf.variable_scope('InceptionResnetV2',
	                               reuse=self._reuse_weights) as scope:
	          return inception_resnet_v2.inception_resnet_v2_base(
	              preprocessed_inputs, final_endpoint='PreAuxLogits',
	              scope=scope, output_stride=self._first_stage_features_stride,
	              align_feature_maps=True)
	
	  def _extract_box_classifier_features(self, proposal_feature_maps, scope):

	    with tf.variable_scope('InceptionResnetV2', reuse=self._reuse_weights):
	      with slim.arg_scope(inception_resnet_v2.inception_resnet_v2_arg_scope(
	          weight_decay=self._weight_decay)):
	        # Forces is_training to False to disable batch norm update.
	        with slim.arg_scope([slim.batch_norm],
	                            is_training=self._train_batch_norm):
	          with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
	                              stride=1, padding='SAME'):
	            with tf.variable_scope('Mixed_7a'):
	              with tf.variable_scope('Branch_0'):
	                tower_conv = slim.conv2d(proposal_feature_maps,
	                                         256, 1, scope='Conv2d_0a_1x1')
	                tower_conv_1 = slim.conv2d(
	                    tower_conv, 384, 3, stride=2,
	                    padding='VALID', scope='Conv2d_1a_3x3')
	              with tf.variable_scope('Branch_1'):
	                tower_conv1 = slim.conv2d(
	                    proposal_feature_maps, 256, 1, scope='Conv2d_0a_1x1')
	                tower_conv1_1 = slim.conv2d(
	                    tower_conv1, 288, 3, stride=2,
	                    padding='VALID', scope='Conv2d_1a_3x3')
	              with tf.variable_scope('Branch_2'):
	                tower_conv2 = slim.conv2d(
	                    proposal_feature_maps, 256, 1, scope='Conv2d_0a_1x1')
	                tower_conv2_1 = slim.conv2d(tower_conv2, 288, 3,
	                                            scope='Conv2d_0b_3x3')
	                tower_conv2_2 = slim.conv2d(
	                    tower_conv2_1, 320, 3, stride=2,
	                    padding='VALID', scope='Conv2d_1a_3x3')
	              with tf.variable_scope('Branch_3'):
	                tower_pool = slim.max_pool2d(
	                    proposal_feature_maps, 3, stride=2, padding='VALID',
	                    scope='MaxPool_1a_3x3')
	              net = tf.concat(
	                  [tower_conv_1, tower_conv1_1, tower_conv2_2, tower_pool], 3)
	            net = slim.repeat(net, 9, inception_resnet_v2.block8, scale=0.20)
	            net = inception_resnet_v2.block8(net, activation_fn=None)
	            proposal_classifier_features = slim.conv2d(
	                net, 1536, 1, scope='Conv2d_7b_1x1')
	        return proposal_classifier_features
	
	  def restore_from_classification_checkpoint_fn(
	      self,
	      first_stage_feature_extractor_scope,
	      second_stage_feature_extractor_scope):

	
	    variables_to_restore = {}
	    for variable in variables_helper.get_global_variables_safely():
	      if variable.op.name.startswith(
	          first_stage_feature_extractor_scope):
	        var_name = variable.op.name.replace(
	            first_stage_feature_extractor_scope + '/', '')
	        variables_to_restore[var_name] = variable
	      if variable.op.name.startswith(
	          second_stage_feature_extractor_scope):
	        var_name = variable.op.name.replace(
	            second_stage_feature_extractor_scope
	            + '/InceptionResnetV2/Repeat', 'InceptionResnetV2/Repeat_2')
	        var_name = var_name.replace(
	            second_stage_feature_extractor_scope + '/', '')
	        variables_to_restore[var_name] = variable
	    return variables_to_restore
